![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Vector Search with Redispy
## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/vector-search/00_redispy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Prepare data

In this examples we will load a list of movie objects with the following attributes: `title`, `rating`, `description`, and `genre`. 

For the vector part of our vector search we will embed the description so that user's can search for movies that best match what they're looking for.

**If you are running this notebook locally**, FYI you may not need to perform this step at all.

In [5]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/vector-search/resources .
!rm -rf temp_repo

Cloning into 'temp_repo'...
remote: Enumerating objects: 204, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 204 (delta 37), reused 24 (delta 24), pack-reused 152
Receiving objects: 100% (204/204), 9.47 MiB | 10.76 MiB/s, done.
Resolving deltas: 100% (64/64), done.
mv: temp_repo/python-recipes/vector-search/resources: No such file or directory


## Packages

In [1]:
# NBVAL_SKIP
%pip install -q redis numpy sentence-transformers

Note: you may need to restart the kernel to use updated packages.


## Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings created from PDF document chunks. **We need to make sure we have a Redis
instance available.

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [2]:
import os

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

### Create redis client

In [3]:
from redis import Redis
client = Redis.from_url(REDIS_URL)

In [4]:
import json

with open("resources/movies.json", 'r') as file:
    movies = json.load(file)

In [5]:
import numpy as np
from sentence_transformers import SentenceTransformer

# load model for embedding our movie descriptions
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def embed_text(model, text):
    return np.array(model.encode(text)).astype(np.float32).tobytes()

/Users/justin.cechmanek/.pyenv/versions/3.11.9/envs/redis-ai-res/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
# Note: convert embedding array to bytes for storage in Redis Hash data type
movie_data = [
    {
        **movie,
        "vector": embed_text(model, movie["description"])
    } for movie in movies
]

In [7]:
movie_data[0]

{'title': 'Explosive Pursuit',
 'genre': 'action',
 'rating': 7,
 'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.',
 'vector': b'\x9bf|=\xa4a\n;\xb6\x91\xb7;*\xcb~\xbd\x07e\xce\xbb\xc9\x16J=G\xa7?=\xcev\x95<g\xfa\x06\xbe\x0fY\xcf=+\x07p=?\xdb\r\xbd\x95\xf2H\xbdre\xc6<N\xdfa=\x958\x16\xbc\xe1\xd3\x13<N\xaa\x1c=\x13\xef\x89<\xd6\xb0-<\x8e\xb2\x9f\xbc_\x0b\xc3\xbd\xa5NR=cl\xf7\xbcP>\x17\xbe\xc0 \x05\xb9&u\xbf<0\xe2b\xba\xd6\xa6\xa8\xbdr\xdc\xec\xbcWc%=\xa6\xe7r\xbb"OG=:(\x85=s@\xa2\xbc/Z\xd0\xbdK%K\xbd\xb1\xed\x94\xbc`\xddH=\xaa&F<\xe0*\xec<\x88\xd8\x8d\xbd\xc5Z\x98<\x13\xa3\xa3=:g3\xbd+\xcd\xbd\xbd\x90$\xf7;\xf8\xf4z=\x01\xb5\x8c=\x8a\x0e\xc6\xbdoI\x90\xbd\x80\x16\xbd;u\xe7\x0c\xbd\xf32\xc9\xbc\x8b\xf8\xbb\xbcP&u\xbb9\x8f\xca<\x07\x80J=\x10\xaf*=\x96OU\xbd\xc9\xf0\x95\xbc\x10\x02\x19=\x12\xf4K<\xc0\xc2\t=L\x83\xac=\x98\xd7\xb8\xbd\xf7\xb5\x9c\xbd9\x85\x18=\x9fd&=73\xf8<\xfb\xf7\x88<\xabv\xf2\xbb%=[\xbd\xdc\xac\xee\xbb2:A\xb

## Define Redis index schema

In [8]:
from redis.commands.search.field import VectorField, TagField, NumericField, TextField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType

index_name = "movies"

schema = (
    VectorField(
        "vector",
        "HNSW",
        {
            "TYPE": "FLOAT32",
            "DIM": 384,
            "DISTANCE_METRIC": "COSINE"
        }
        ),
        NumericField("rating"),
        TagField("genre"),
        TextField("title"),
        TextField("description")
)

try:
    client.ft(index_name).info()
    print("Index exists!")
except:
    # index Definition
    definition = IndexDefinition(index_type=IndexType.HASH)

    # create Index
    client.ft(index_name).create_index(fields=schema, definition=definition)

## Populate index

In [9]:
def load_docs(client: Redis, data: list[dict]):
    for i, d in enumerate(data):
        client.hset(
            i,
            mapping = d
        )

def print_results(res):
    docs = [(doc.title, doc.genre, doc.rating) for doc in res.docs]
    print(f"Top {len(docs)} movies: ", docs)

In [10]:
load_docs(client, movie_data)

In [11]:
res = client.ft(index_name).search("*")
res

Result{200 total, docs: [Document {'id': 'doc:ragas_ex:e5daf787a0fc4d0b9f29e1996245b4c0', 'payload': None, 'source': 'resources/nke-10k-2023.pdf', 'content': "ASIA PACIFIC & LATIN AMERICA\n\n(1)\n\nGLOBAL BRAND DIVISIONS\n\nTOTAL NIKE BRAND\n\nCONVERSE CORPORATE\n\nTOTAL NIKE, INC.\n\nRevenues by:\n\nFootwear Apparel Equipment\n\n$\n\n11,644 $ 5,028 507\n\n6,970 $ 3,996 490\n\n5,748 $ 2,347 195\n\n3,659 $ 1,494 190\n\n— $ — —\n\n28,021 $ 12,865 1,382\n\n1,986 $ 104 29\n\n— $ — —\n\n30,007 12,969 1,411\n\nOther\n\nTOTAL REVENUES\n\n$\n\n—\n\n17,179 $\n\n—\n\n11,456 $\n\n— 8,290 $\n\n— 5,343 $\n\n25 25 $\n\n25\n\n42,293 $\n\n86 2,205 $\n\n40 40 $\n\n151 44,538\n\nRevenues by:\n\nSales to Wholesale Customers $\n\n10,186 $\n\n7,812 $\n\n4,513 $\n\n3,387 $\n\n— $\n\n25,898 $\n\n1,353 $\n\n— $\n\n27,251\n\nSales through Direct to Consumer Other\n\n6,993 —\n\n3,644 —\n\n3,777 —\n\n1,956 —\n\n— 25\n\n16,370 25\n\n766 86\n\n— 40\n\n17,136 151\n\nTOTAL REVENUES\n\n$\n\n17,179 $\n\n11,456 $\n\n8,

## Index loaded now we can perform vector search

### basic vector search

In [12]:
from redis.commands.search.query import Query

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

# Note: dialect 2 and above required for vector search
query = Query("(*)=>[KNN 3 @vector $vec_param AS dist]").sort_by("dist").dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)


Top 3 movies:  [('Fast & Furious 9', 'action', '6'), ('Despicable Me', 'comedy', '7'), ('The Incredibles', 'comedy', '8')]


### Hybrid filter vector search

Redis allows you to combine filter searches on fields within the index object allowing us to create more specific searches.

In [13]:
# Search for top 3 movies specifically in the action genre

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

# Note: genre is a tag field in our schema so the syntax is @<field_name>:{ <tag> | <tag> | ...}
query = Query("(@genre:{action})=>[KNN 3 @vector $vec_param AS dist]").sort_by('dist').dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 3 movies:  [('Fast & Furious 9', 'action', '6'), ('Mad Max: Fury Road', 'action', '8'), ('Explosive Pursuit', 'action', '7')]


In [14]:
# Search for top 3 movies specifically in the action genre with ratings at or above a 7

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

query = Query("(@genre:{action} & (@rating:[7 inf]))=>[KNN 3 @vector $vec_param AS dist]").sort_by('dist').dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 3 movies:  [('Mad Max: Fury Road', 'action', '8'), ('Explosive Pursuit', 'action', '7'), ('The Avengers', 'action', '8')]


In [15]:
# Search with full text search for movies that directly mention "criminal mastermind" in the description

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

query = Query("(@description:(criminal mastermind))=>[KNN 3 @vector $vec_param AS dist]").sort_by('dist').dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 2 movies:  [('Despicable Me', 'comedy', '7'), ('The Dark Knight', 'action', '9')]


In [16]:
# Vector search with wild card match

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

query = Query("(@description:(crim*))=>[KNN 3 @vector $vec_param AS dist]").sort_by('dist').dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 3 movies:  [('Despicable Me', 'comedy', '7'), ('The Incredibles', 'comedy', '8'), ('Explosive Pursuit', 'action', '7')]


In [17]:
# Vector search with fuzzy match

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

# Note: fuzzy match is based on Levenshtein distance. Therefore, "hero" might return result for "her" as an example.
# See docs for more info https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/query_syntax/
query = Query("(@description:%hero%)=>[KNN 3 @vector $vec_param AS dist]").sort_by('dist').dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 3 movies:  [('The Avengers', 'action', '8'), ('Black Widow', 'action', '7'), ('The Princess Diaries', 'comedy', '6')]


## Range queries

Range queries allow you to set a pre defined "threshold" for which we want to return documents. This is helpful when you only want documents with a certain distance from the search query.

In [18]:
user_query = "Family friendly fantasy movies"

embedded_user_query = embed_text(model, user_query)

query = (
    Query("@vector:[VECTOR_RANGE $radius $vector]=>{$YIELD_DISTANCE_AS: vector_distance}")
     .sort_by("vector_distance")
     .return_fields("title", "rating", "genre", "vector_distance")
     .dialect(2)
)

# Find all vectors within 0.8 of the query vector
query_params = {
    "radius": 0.8,
    "vector": embedded_user_query
}

res = client.ft(index_name).search(query, query_params)
print_results(res)


Top 6 movies:  [('The Incredibles', 'comedy', '8'), ('Black Widow', 'action', '7'), ('Despicable Me', 'comedy', '7'), ('Shrek', 'comedy', '8'), ('Monsters, Inc.', 'comedy', '8'), ('Aladdin', 'comedy', '8')]


Like the queries above, we can also chain additional filters and conditional operators with range queries. The following adds an `or` condition that returns vector search within the defined range or with a rating at or above 9.

In [19]:
user_query = "Family friendly fantasy movies"

embedded_user_query = embed_text(model, user_query)

query = (
    Query("@rating:[9 +inf] | @vector:[VECTOR_RANGE $radius $vector]=>{$YIELD_DISTANCE_AS: vector_distance}")
     .sort_by("vector_distance")
     .return_fields("title", "rating", "genre", "vector_distance")
     .dialect(2)
)

# Find all vectors within 0.8 of the query vector
query_params = {
    "radius": 0.7,
    "vector": embedded_user_query
}

res = client.ft(index_name).search(query, query_params)
print_results(res)

Top 3 movies:  [('The Incredibles', 'comedy', '8'), ('The Dark Knight', 'action', '9'), ('Inception', 'action', '9')]


In [20]:
# clean up!
client.flushall()

True